In [1]:
import numpy as np
import pandas as pd
from calendar import isleap

In [13]:
#constant parameters values
tabs0=273.16
E2=0.01 
secd=86400.

'''  
and yet another input of the physical constants
[cp]=[J/kg/K], 
[RV]=[J/kg/K;m**2/s**2/k], 
[sig]=[W/m**2/grad**4],
[L]=[J/kg],[G]=[m/s**2]
lpl  - (kal/g) specific heat of ice fusion (plavleniya)
kinv - (m**2/s) coefficient of kinematic vyazkosti
zcb  - (kal/g/grad) cpecific heat capacity of water
zrb  - density of water (g/cm^3) 
cic  - specific heat capacity of ice (kal/g/grad)         
RL - ice density (g/cm^3);
'''
cp,rv,kapa,sig,L,g,LPL,kinv,zcb,zrb,cic,rl = 1005.,287.05,0.43,0.000000057,2501000.,9.81,80.,0.000013,1.,1.,0.5,0.9

# months lenght
MON = [31,28,31,30,31,30,31,31,30,31,30,31]

# input of the data for the calculation of G by Budyko
DBMES_N = [-0.23,-0.15,0.08,0.15,0.23,0.23,0.19,0.12,-0.08,-0.12,-0.19,-0.23]

# for the northern hemisphere
DBMES_S = [0.19,0.12,-0.08,-0.12,-0.19,-0.23,-0.23,-0.15,0.08,0.15,0.23,0.23]

# timestep parameters
tstep=24.0          # time_step (24 hour)
nstep=24/tstep      # number of time steps in a day
deltat=tstep*3600  # time_step in sec. 

# parameters  which are the same for all grid cells
level0=2.          # the reference height for atmospheric forcings (air temperature and air humidity) (m)
zwind0=10.         # the reference height of wind speed measurements (m)
rs0=0.15           # snowpack density for fresh snow (g/cm^3)
ser=1.             # coefficient of "serosti"
zcrt=0.22          # (kal/g/grad) specific heat capacity of dry soil
zerot1=290.        # transition from positive to negative air temperatures at autumn (number of day)
ev=1               # parameter for the calculation of transpiration
hpod=0.            # thickness of podstilki
Lpod=0.00015       # TEPLOPROVODNOST' PODSTILKI
emkl=0.1           # coefficient for the calculation of interception capacity for liquid precipitations
tayl=3.            # for the calculation of water table depth  (in days)
dexp=0.2           # parameter for heat exchange in the forest
exp2=1.            # parameter for the forest

# yet another parameters provided
mmm=TAYL*nstep    # for the water table depth calc (in days*nstep)
sncover=1.        # maybe snow cover correction factor

In [14]:
# define calculation periods
start_date = pd.to_datetime('1980-07-01') # format 'year-month-day'
end_date   = pd.to_datetime('2012-12-31') # format 'year-month-day'

# calculate the first day of calculations (day of year)
firday = start_date.dayofyear

# define list of the years for calculation
years = [i for i in range(start_date.year, end_date.year+1)]

#define the river name
river = '70194'

#define number of cells in basin we calculated
ncell_bas = 1

#initial guess for calibration parameters
opt_par = np.ones(20)

In [15]:
path_fixed_fields = '../fixed_fields/fixed_param_cl_' + river + '.txt'
path_monthly_fields = '../monthly_fields/monthly_param_' + river + '.txt'
path_forcing_data = '../forcing_data/F' + river + '.dat'

In [30]:
# fuction for air density (in kg/m^3) calculation
def FUNRO(XX, YY, ZZ):
    return ZZ*100./(rv*XX*(1.+0.61*YY))

In [17]:
# refers to the lines 1490 - 1522
# function which convert monthly parameters to daily
def monthly_to_dayly(values, year): 
    values = np.array(values)
    # one additional year in both directions for proper interpolation
    mon_indexes = pd.period_range(start=start_date.year-1, end=end_date.year+2, freq='M')[:-1]
    mon_values = values.tolist()*int(len(mon_indexes)/12)
    daily_values = pd.DataFrame({'data': mon_values}, index=mon_indexes).resample('D').shift(14).interpolate(method='linear')
    daily_values = daily_values[str(year)].values
    return daily_values

In [19]:
# TSAR-CYCLE starts here!!!
for icell in range(1, ncell_bas+1):
    
    # read parameters specified for each model cell
    ncl,ico,jco,lon,lat,ind1,area,sand,clay,amt,sryt,albzim, \
    prles,sai,leaf,emkh,xi,yi,hroot,h0,hveg,elevat,ksiot0,d0 = np.loadtxt(path_fixed_fields)
    
    # read monthly parameters for each cell
    monthly_params = np.loadtxt(path_monthly_fields, delimiter=',')
    land     = monthly_params[0]
    albedo_m = monthly_params[1:13]
    grnfr_m  = monthly_params[13:25]
    z0veg_m  = monthly_params[25:37]
    lai_m    = monthly_params[37:49]
    eftr_m   = monthly_params[49:61]
    
    # parameters calculation for each cell (Cosby scheme)
    bpar = 0.157*clay*100 - 0.003*sand*100 + 3.1                                             # Cosby
    fi0  = -(10**( -0.0095*sand*100 + 0.0063*(100 - sand*100 - clay*100) + 1.54 )/100.)      # Cosby
    por  = (-0.037*clay*100 - 0.142*sand*100 + 50.5)/100                                     # Cosby
    k0   = ( 10**( -0.0064*clay*100.+0.0126*sand*100.-0.6 )/60/60 )*2.54*0.01                # log Cosby trans to m/s
    nb   = por*(0.1/1000./24./60./60./k0)**(1/(2*bpar+3))                                    # fc_Cosby(under k=0.1mm/day)
    wzav = por*(-150/fi0)**(-1/bpar)                                                         # wp_Cosby(fi=-150m)
    
    '''
    if (kod_cal.ne.1):
    k0_opt=opt_par(3)
    k0=k0*exp(k0_opt)
    '''
    
    d00_m = np.ones(12)*d0
    
    '''
    if (kod_cal.ne.1) then
    albedo_m(i)=albedo_m(i)*opt_par(6)
    LAI_M(I)=LAI_M(I)*opt_par(7)
    endif
    '''
    
    if sryt < -1:
        
    
    # READING values of FORCING factors
    # precipitatiom - kg/m^2/s^1=mm/s; 
    # radiation - W/m**2;
    # temperature -K;
    # wind speed - m/s at 10 m; 
    # pressure - Pa; 
    # air specific humidity - kg/kg
    forcings = np.loadtxt(path_forcing_data)
    sw     = forcings[:, 0]
    rdown  = forcings[:, 1]
    rainf  = forcings[:, 2]
    snowf  = forcings[:, 3]
    t45    = forcings[:, 4]
    u45    = forcings[:, 5]
    pres   = forcings[:, 6]
    q45    = forcings[:, 7]
    
    #coefficients to forcings (!!!) - refer to opt-par array later
    krain=1
    ksnow=1
    k_sw=1
    k_lw=1
    
    # FORCINGS TRANSFORMATION
    #radiation correction
    rdown  = rdown*k_lw
    sw     = sw*k_sw
    # total precipitation correction
    totalpr= rainf+snowf
    totalpr[t45-tabs0 < 0]  *= ksnow
    totalpr[t45-tabs0 >= 0] *= krain 
    # wind speed at 10 m correction
    u45 = np.where(u45 < 0.5, 0.5, u45)
    # translation of pressure from Pa into gPa=mb
    pres = pres/100
    # translation of precipitation from mm/s into mm/time_step
    totalpr = totalpr*deltat
    
    # some strange assignment
    pr=totalpr   
    
    # Yet Another deal with forcings (was placed in a SROK cycle, 
    # but i think it is not a good idea do the same transformation for each srok)
    # row 817-820 in fortran version
    # no information provided about a3 and a4 - some pressure-temperature related arrays
    tc45 = t45 - tabs0  
    a3 = (0.623 * 6.1 / pres) * np.exp(17.1 * tc45 / (235 + tc45))
    a4 = a3 * 17.1 * 235 / (235 + tc45) ** 2
    a5 = (1000. / pres) ** .288 # row 1552 in fortran version
    
    '''
    if(sryt.lt.-1.) then
    GOTO  1879 (continue)    
    else  

    krain=1
    ksnow=1
    k_sw=1
    k_lw=1

    if (kod_cal.ne.1) then
    hroot=hroot*opt_par(1)
    h0=hroot*opt_par(2)

    krain=opt_par(8)
    ksnow=opt_par(9)
    k_sw=opt_par(10)
    k_lw=opt_par(11)
    endif
    
    endif
    '''
    
    '''
    c h0 - the depth of unpermeable layer (m)
    c NB - field capacity
    c WZAV - wilting point;   
    C POR - porosity (m^3/m^3)
    C BPAR - Clapp-Hornberger b parameter;
    C K0 - hydraulic conductivity at saturation (m/s);
    c FI0 - saturated matric potential (m)
    c HROOT - root-zone depth (m);
    c Z0VEGm - roughness coefficient of vegetation (m); 
    c LAI_M = total_LAI  
    c grnfr_m - greenness fraction (green_LAI/total_LAI 
    C d00 - zero displacement height  (m)
    c albedo_m - snow_free albedo
    c hveg - the height of vegetation (m)
    C AMT - amlitude of air temperature (in C),
    c SRYT - soil temperature at the depth where seasonal variations are absent (in C);
    C LEAF -size of grass's leaves in (cm)
    c WH - min. amount of unfrozen water
        '''
    
    # calculation of zerot1 (in days from 1 January)
    if lat >= 0:
        zerot1=373.+1.914*lat-0.05682*lat*lat-0.03234*elevat
        if zerot1 >= 365:
            zerot1=365
        elif zerot1 < 0:
            zerot1=0
    else: # lat<0
        zerot1=662.+14.06*lat+0.102*lat*lat-0.0604*elevat+0.0000015*elevat*elevat   
        if zerot1 < 0:
            zerot1=0
        elif zerot1 > firday:
            zerot1=firday
    
    # Z0veg_m preprocessing
    z0veg_m = np.where(z0veg_m == 0, 0.0024, z0veg_m)
    
    if prles == 0:
        emkh       = 0
        sai        = 0
        hveg       = 0.2
        d00_m[:]   = 0
        z0veg_m[:] = hveg*2/3
    
    # transformation of input parameters
    if por-nb < 0.1:
        nb = por - 0.1
    if sryt < 0:
        sryt = 0 # for the first approximation for the territories with permafrost (really there should be another program)
    
    hroot = hroot*1000      # translation from m to mm
    h0 = h0*1000.           # translation from m to mm
    
    # CALCULATION OF the rest MODEL PARAMETERS:
    level = level0+hveg      # m
    zwind = zwind0+hveg      # m
    vdhgr = por-nb           # water yield of groundwater
    
    # calculation of TRMN=ET00/EP00  
    if leaf == 0:
        trmn = 0
    else:
        trmn = 0.825*0.5/np.sqrt(leaf)+0.784
    
    # calculation of soil parameters
    # 2.65 - density of hard fraction of the soil (g/cm^3) 
    # ZRT  - density of dry soil (g/cm^3)
    zrt = (1 - por)*2.65
    
    # translation of FI0 into positive value
    fi0 = -1*fi0
    
    # calculation by Kalyuzhnyi and Pavlova
    # UMG - maximum hygroscopicity
    # USS - static water
    wh  = (nb+wzav)/2
    uss = wzav/1.35
    umg = uss 
    lc  = (0.102*np.exp(4.7*umg)+0.45*zrt-0.35)*0.0024
    
    # calculation of standard deviation of saturated hydraulic conductivity (SIGK0) for the calculation of runoff
    # translation of m/s into mm/min
    k0    = k0*60*1000                        
    sigk0 = 0.9*k0**0.6                 
    if np.sqrt(3)*sigk0 > k0:
        sigk0 = k0/np.sqrt(3)
    
    # translation of mm/min into mm/hour
    k0    = k0 * 60
    sigk0 = sigk0 * 60
    
    # calculation of the effective capilar. potential HK (mm)
    x1 = 1 - (2 + 3/bpar)
    x2 = fi0 ** x1
    hk = (-x2 * fi0 ** x1 / x1) * 1000
    
    #ACCOUNTING FOR  SOUTHERN OR NORTHERN hemisphere (for the calculation of G by Budyko)
    if lat < 0: 
        dbmes = DBMES_S
    else:
        dbmes = DBMES_N
    
    # set initial conditions
    w0     = nb
    w1m    = nb
    w2     = w0
    uzim   = w0
    ice    = 0  
    psn    = 0
    hsnow  = 0
    ksi    = 0                        
    ksiot  = 0
    ice1   = 0
    u1     = w0
    sumsn  = 0
    wlsn   = 0   
    snles  = 0
    wles   = 0  
    interpr= 0
    t2daypr=tabs0 + 5
    t0daypr=tabs0 + 5  
    sumhyd = 0
    hsr    = sumhyd/mmm   # mmm=TAYL*nstep  - for the water table depth calc (in days*nstep)
    h2soil = h0
    h2psoil= h2soil
    hhyd0  = np.ones(mmm)*0.0000000001
    kodhyd2= 0
    k80    = 0
    hyd1   = 1.e-5
    hyd2   = 1.e-5
    hyd11  = 1.e-5
    hyd22  = 1.e-5
    hyd1pr = 1.e-5
    tq     = -(deltat/3600/2)
    nhour  = 0
    prhour = 0 # in fortran code prhour = nhour
    sumtp  = 0
    sumtc  = 0
    sumqbz = 0 
    nleto  = 0
    nzima  = 0
    kodzero= 0               # if =1, then zerot1 is not changed; else (=0) zerot1 should be recalculated    
    L1 = 0.0049 * rs0 ** 2   # formula by Bracht
    
    
    # start of the YEAR CYCLE
    for year in years:
        if isleap(year) == True:
            days_in_year = 366
            MON[1] = 29
        else:
            days_in_year = 365
            MON[1] = 28
        
        if year == start_date.year:
            jdy = firday
        else:
            jdy = 1
    
        '''
        Indexes in fortran starts from 1
        but in python - from 0
        don't forget (!!!!!!) SUBSTRACT 1 from fortran array indexes! (index-1)
        '''

        # calculation of daily values of LAI, etc:
        sdbmes   = monthly_to_dayly(dbmes, year)
        laid    = monthly_to_dayly(lai_m, year)
        grnfrd  = monthly_to_dayly(grnfr_m, year)
        eftrd  = monthly_to_dayly(eftr_m, year)
        z0vegd  = monthly_to_dayly(z0veg_m, year)
        d00d  = monthly_to_dayly(d00_m, year)
        albedod  = monthly_to_dayly(albedo_m, year)

        # start of the DAY CYCLE
        for jdy in range(jdy, days_in_year+1):

            # initial conditions for the year
            k_snowt    = 0          #int
            swnet_d    = 0          #float
            lwnet_d    = 0
            qle_d      = 0
            qh_d       = 0
            qg_d       = 0
            qf_d       = 0
            qv_d       = 0
            snowt_d    = 0
            vegt_d     = 0
            avgt_d     = 0
            radt_d     = 0
            t2_d       = 0
            t0_d       = 0
            precip_d   = 0
            hprecp_d   = 0 
            ec_d       = 0   
            et_d       = 0
            ep_d       = 0
            inter_d    = 0
            stok_d     = 0
            drain_d    = 0
            surrun_transf = 0
            drain_transf  = 0
            snow_d     = 0
            WK_d       = 0
            esnow_d    = 0
            albedo_d   = 0
            swe_d      = 0
            snmlt_d    = 0
            snowfr_d   = 0
            ksi_d      = 0
            ksiot_d    = 0   
            w2_d       = 0 
            vd_d       = 0
            hgr_d      = 0
            sweveg_d   = 0
            dvsoil_d   = 0
            dswe_d     = 0
            dint_d     = 0
            soilw_d    = 0
            epot_d     = 0
            subsl_d    = 0
            snfr_d     = 0
            snalb_d    = 0
            canint_d   = 0
            radtmax    = 0
            radtmin    = 1000

            # calculation of the ground heat flux by Budyko
            # relation between annual amplitude of air temperature and ...?
            amb = (0.184 * amt * 1000 / 30.4)/2

            # calculation of the ground heat flux B (cal/cm**2/day)
            b = sdbmes[jdy-1] * amb

            # transformation of (cal/cm**2/day) into (W/m**2)
            b = b / 1440 * 697.37

            # time-step value of LAI, LSAI, EF, Z0VEG, D00, ALBLET:    
            lai   = laid[jdy-1]*grnfrd[jdy-1]   # calculation of green_LAI
            lsai  = laid[jdy-1]+sai             # total_LAI + SAI
            ef    = eftrd[jdy-1]  
            z0veg = z0vegd[jdy-1]
            d00   = d00d[jdy-1]
            alblet= albedod[jdy-1]

            # so strange assingment
            bbb=b

            # cycles that count ZIMA and LETO

            if t2daypr > tabs0:
                nleto = nleto+1
            else:
                nleto = 0

            if t2daypr <= tabs0:
                nzima = nzima+1
            else:
                nzima = 0

            # start TIMESTAMP CYCLE
            for step in range(1, int(nstep)+1):
                
                if sumsn+wlsn == 0:
                    rspr = rs0
                if snles+wles == 0:
                    rslespr = rs0
                
                inter2 = 0  # liquid water on the canopy at the end of time step 
                            #(after evaporation of intercepted precipitation) (in LETO)
                inter  = 0  # evaporation of intercepted precipitation during the warm season (in LETO)
                kond   = 0
                
                # one another strange assignment
                b=bbb
                
                # calculation of soil parameters which depend on soil moisture
                # CALCULATION BY (3.11), P.111 FROM Kalyuzhnyi,Palova,Lavrov (W/m/grad)           
                L3 = 0.102*np.exp(4.7*w1m) + 0.45*zrt-0.35    # teploprovodnost' of unfrozen soil 
                L3 = L3*0.0024                                #  translation into kal/cm/c/grad                                                        
                zc3 = zcrt * zrt + zcb * zrb * w1m
                
                # WH5 - the amount of unfrozen water under the soil temperature =-5C (Kalyuzhnyi and Pavlova)  
                wh5 = 0.94*wzav + 0.017*zrt
                zc2 = zcb*uzim*zrb + cic*ice*rl + zcrt*zrt + (wh-wh5)*LPL*zrb/5
                L2  = 1.15 * L3                                  
                za3 = L3 / zc3
                
                # determination of KODZIM. 
                # IF KODZIM=1 then ZIMA-subprogram is used,
                # ELSE (KODZIM=0)  LETO-subprogram is used
                if jdy <= 210:
                    if (hsnow == 0) & (ksi == 0) & (nzima <= 7 & nleto >= 1):
                        kodzim=0
                    else:
                        kodzim=1
                else:
                    if (((t2daypr < tabs0) | (hsnow > 0) | (ksi > 0) | (nzima >= 7)) & (nleto <= 1)) | hsnow >0 | ksi > 0:
                        kodzim=1
                    else:
                        kodzim=0
                
                # calculation of albedo, sumsn - is  snow pack (mm)         
                # variant 1: albsnow=0.834-21.988*(rspr-0.1)**3 
                # variant 2: albsnow=0.8-21.988*(rspr-0.1)**3
                # will search a parameter optima
                albsnow = opt_par[6] - 21.988*(rspr-0.1)**3
                if albsnow < 0.1:
                    albsnow = 0.1
                
                if (sumsn+wlsn) < 1:
                    albedo = alblet + (albsnow - alblet) * np.sqrt((sumsn+wlsn)*0.1)
                else:
                    albedo = albsnow
                
                # calculate the radiation
                if kodzim == 1:
                    exples = np.exp(-prles*lsai)
                    radles = sw*(1 - albzim)+rdown # it is an array here
                    rad    =(sw*(1 - albedo)+rdown)*exples
                else:
                    exples = 1
                    albedo = alblet
                    rad = sw * (1 - albedo) + rdown
                    radles = 0 # but here it is a real
                
                if kodzim == 1:
                    zimw = 1/(1+8*ice1)**2 * (u1-wzav)/(2/3*nb)
                
                """
                CALL ITBLOCK(KEYITER,HSNOW,KSI,KSIOT,A3,A4,
                D00,LEVEL,TABS0,B,EFIZL,P,U2,T2,Q2,T0,G,EP0,
                zwind,e2,WLSN,kodzim,radles,tles,LSAI,expLES,d,
                MLES,ELES,WLES,SNLES,DEXP,exp2,z0veg,ef,ev,trles,LPOD,HPOD,
                zimw,HL,BZV,kodsubp,kodsubl)
                """
                

/projects/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: FutureWarning: .resample() is now a deferred operation
use .resample(...).mean() instead of .resample(...)


In [31]:
#block of iterations inside TIMESTAMP loop
def itblock():
    """
    keyiter, hsnow, ksi, ksiot, a3, a4, d00,
            level, tabs0, b, efizl, p, u2, t2, q2, t0, g, ep0,
            zwind, e2, wlsn, kodzim, radles, tles, lsai, exples, 
            d, mles, eles, wles, snles, dexp, exp2, z00, ef, ev,
            trles, Lpod, hpod,zimw, hl, bzv, kodsubp, kodsubl
    """
    # Setting up initial (at the first iteration) values  
    # cles - teploemkost' drevesiny 2700 J/kg/grad; biomass=2x10^5 kg/ga=20 kg/m^2
    cles = 3000*40/deltat
    
    # Le is not defined in a code above
    Lcan, L = Le
    
    kodsubp=0
    kodsubl=0
    if snles > 0:
        Lcan    = Le+LPL*4.19*1000
        kodsubl = 1
    if hsnow > 0 | (ksi > 0 & ksiot ==0): 
        L       = Le+LPL*4.19*1000
        kodsubp = 1
    
    ro = FUNRO(t45, q45, pres)
    
    t0, tles, tp = t45
    t00pr        = t0
    tlespr       = tles
    
    # radsur is not define in a code above
    rad         = radsur
    
    eles  = 0
    trles = 0
    mles  = 0  
    bles  = 0
    
    # ff for T and q at 2 m, FFu2 and ff10 for wind speed at 2 and 10 m
    ff10 = np.log((zwind - d00) / z00)
    ff   = np.log((level - d00) / z00)
    ffu2 = np.log((level - d00) / z00)  
    if kodzim == 1 & exples != 1:
        les  = 1
        tles = t0
        keykod = 1
    else:
        les  = 0
        keykod=1000   # there is no forest   
        hl   = 0
        bzv  = 0        
    
    keyiter = 0
    
    # MAIN iterations starts here
    # label 3747
    if kodzim == 1:
        if les == 1:
            tp  = t0
            rad = radles
            bles= -sig*ser*(4*t45**3*tp-3*t45**4)
        else:
            hl  = -kapa*cp*ro*uzv*tzv
            tles= t0
            rad = radsur
            efizles = sig*ser*exp2*(1-exples)*(4*t45**3*tles-3*t45**4)
            rad = rad + efizles
        
    sumff10 = ff10
    sunff   = ff
    sumffu2 = ffu2
    
    ITER = 1
    
    # calculation of the dynamical speed
    # label 4300   
    uzv = u45*kapa/ff10 
    u2  = uzv*ffu2/kapa  
    d   = 0.0076*u2*np.exp(-dexp*lsai)/(1+0.82*np.sqrt(u2))
    if les == 1: 
        # goto label 4004
    if kodzim ==

    
    return ro #cles

In [15]:
#lines 581-610 nothing clear, put in future bag

In [35]:
def abc():
    abc = a2
    return abc
print(abc())
a2 = 5

NameError: name 'a2' is not defined

In [11]:
pd.period_range(start=start_date, end=end_date, freq='A')

PeriodIndex(['1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
             '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
             '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
             '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
             '2012'],
            dtype='int64', freq='A-DEC')

In [9]:
a = np.arange(-5, 5)

In [10]:
np.exp(17.1 * a / (235. + a))

array([ 0.6895341 ,  0.74371016,  0.80161993,  0.86348296,  0.92952933,
        1.        ,  1.07514725,  1.15523501,  1.24053932,  1.33134867])

In [28]:
a[a>0] *= 100

In [29]:
a

array([ -5,  -4,  -3,  -2,  -1,   0, 100, 200, 300, 400])

In [22]:
a = 5
def re():
    a = 7
    return a
a = re()
a

7